In [3]:
import sys
import os
import torch
import torch.onnx
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
import tensorflow as tf
from torch.autograd import Variable
from onnx_tf.backend import prepare
import onnxruntime as ort
import torchvision.transforms as transforms
from torch.utils.data import DataLoader as dataloader
from torchvision import datasets
from torchvision import models
from PIL import Image
import numpy as np
import onnx
sys.path.insert(1, 'T:/Haematology FYP/HelperScripts')
from HelperFunctions import *
import keras

In [4]:
device = torch.device(0 if torch.cuda.is_available() else 'cpu')
device_name = torch.cuda.get_device_name(0)

In [5]:
model = models.resnet18(pretrained=True).to(device)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5).to(device)
model_file = "T:/Haematology FYP/Models/baseline/temp/Greenresnet18_profiling_Green.pt"
check_point = torch.load(model_file)
model.load_state_dict(check_point['model_state_dict'])

<All keys matched successfully>

In [6]:
# Export the trained model to ONNX
input = Variable(torch.randn(64, 3, 160, 160).to(device))
model.eval()
torch.onnx.export(model, input, "T:/Haematology FYP/Models/resnet18_new.onnx", input_names=['input'],output_names=['output'])

In [5]:
classes = ['Basophil','Eosinophil','Lymphocyte','Monocyte','Neutrophil']
loader, _ = datamaker('Green','T:/Haematology FYP/Data/WBC_Classification_3172_c/',batch_size=64)
img, true = next(iter(loader))
img = img.numpy()
model = ort.InferenceSession("T:/Haematology FYP/Models/resnet18_new.onnx")
input = {'input':img}
output = model.run(None,input)[0]
preds = torch.from_numpy(output.argmax(axis=1))
correct = preds.eq(true.view_as(preds)).sum()
acc = correct.float()/preds.shape[0]
print(acc)

Train samples: 4563    Batch size: 71
Validation samples: 782     Batch size: 12


C:\Users\zarre\.conda\envs\gpu\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:350: UserWarning: Deprecation warning. This ORT build has ['CUDAExecutionProvider', 'CPUExecutionProvider'] enabled. The next release (ORT 1.10) will require explicitly setting the providers parameter (as opposed to the current behavior of providers getting set/registered by default based on the build flags) when instantiating InferenceSession.For example, onnxruntime.InferenceSession(..., providers=["CUDAExecutionProvider"], ...)
  warnings.warn("Deprecation warning. This ORT build has {} enabled. ".format(available_providers) +


tensor(0.9062)


In [7]:
# Load the ONNX file
model = onnx.load("T:/Haematology FYP/Models/resnet18_new.onnx")

In [8]:
# Import the ONNX model to Tensorflow
tf_rep = prepare(model, device='cpu') 

In [9]:
loader, _ = datamaker('Green','T:/Haematology FYP/Data/WBC_Classification_3172_c/',batch_size=1)
img, true = next(iter(loader))
img = img.numpy()

output = tf_rep.run(img)
preds = torch.from_numpy(output[0].argmax(axis=1))
correct = preds.eq(true.view_as(preds)).sum()
acc = correct.float()/preds.shape[0]
print(acc)
tf_rep.export_graph('T:/Haematology FYP/Models/resnet18_new')


Train samples: 4563    Batch size: 4563
Validation samples: 782     Batch size: 782
tensor(1.)


INFO:tensorflow:Assets written to: T:/Haematology FYP/Models/resnet18_new\assets


INFO:tensorflow:Assets written to: T:/Haematology FYP/Models/resnet18_new\assets


In [10]:
#new_model = tf.saved_model.load('T:/Haematology FYP/Models/resnet18_new')
converter = tf.lite.TFLiteConverter.from_saved_model('T:/Haematology FYP/Models/resnet18_new')

In [11]:
def rep_dataset():
    """Generator function to produce representative dataset for post-training quantization."""

    # Use a few samples from the training set.
    for _ in range(100):
        img = iter(loader).next()[0].numpy()
        img = [(img.astype(np.float32))]
        yield img

In [12]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = rep_dataset
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model = converter.convert()

In [13]:
open('model.tflite', 'wb').write(tflite_model)

11324152

In [ ]:
# Load the TFLite model in TFLite Interpreter
interpreter = tf.lite.Interpreter('model.tflite')

interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = img
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data.shape)

Error: Session cannot generate requests

In [ ]:
print(img.shape)
output = tflmodel(img[0])

Error: Session cannot generate requests

In [14]:
new_model = keras.models.load_model('T:/Haematology FYP/Models/resnet18_new')

In [19]:
infer = new_model.signatures["serving_default"]


In [20]:
print(infer)

ConcreteFunction signature_wrapper(*, input)
  Args:
    input: float32 Tensor, shape=(64, 3, 160, 160)
  Returns:
    {'output': <1>}
      <1>: float32 Tensor, shape=(64, 5)
